## ■ Image Files resize (32x32)

In [1]:
import  cv2
import  os 
import  numpy  as np

path = "E:\\data11\\original"

file_list = os.listdir(path)
    
for k in file_list:
    img = cv2.imread(path + '\\' + k)
    width, height = img.shape[:2]
    resize_img = cv2.resize(img, (32 , 32), interpolation=cv2.INTER_CUBIC)
    cv2.imwrite('E:\\data11\\resize\\' + k, resize_img)

## ■ Setting

#### ▶ Train_data (1 ~ 7000) :  7000개

#### ▶ Test_data (7001 ~ 7470) : 470개

## ■ Create Labels

#### ● Change 2 dimension array to 1 dimension array

In [2]:
import csv

path1 = 'E:\\data11\\xray_labels.csv'
file = open(path1)

labeldata = csv.reader(file)
labellist = []

for  i   in  labeldata:
    labellist.append(i)

labellist2 = sum(labellist,[])

#### ● Change 'normal' = 1 , 'patient' = 0

In [3]:
path2 = 'E:\\data11\\xray_labels2.csv'
file2 = open(path2, 'w')

# normal = 1 , patient = 0

for  i  in  labellist2:
    if i == 'patient':
        file2.write( str(0) + '\n' )
    else:
        file2.write( str(1) + '\n' )
        
file2.close()

#### ● Save Train Labels and Test Labels

In [4]:
# save train labels

path = 'E:\\data11\\xray_labels2.csv'
file = open(path)

labeldata = csv.reader(file)
labellist = []

for  i   in  labeldata:
    labellist.append(i)

labellist2 = sum(labellist,[]) # sum 함수의 default 값은 0이므로 []로 바꾸어서 리스트 합침.

# print(labellist)


path2 = 'E:\\data11\\train_label.csv'
file2 = open(path2, 'w')

for  i  in  range(0,7000):
    file2.write( str(labellist2[i]) + '\n' )
    
file2.close()

In [5]:
# save test labels

path = 'E:\\data11\\xray_labels2.csv'
file = open(path)

labeldata = csv.reader(file)
labellist = []

for  i   in  labeldata:
    labellist.append(i)

labellist2 = sum(labellist,[]) # sum 함수의 default 값은 0이므로 []로 바꾸어서 리스트 합침.

# print(labellist)


path2 = 'E:\\data11\\test_label.csv'
file2 = open(path2, 'w')

for  i  in  range(7000,7470):
    file2.write( str(labellist2[i]) + '\n' )
    
file2.close()

## ■ Create loader4.py 

 ■ image_load  
 ■ next_batch  
 ■ shuffle_batch  
 ■ label_load  

In [6]:
import   csv
import  os
import  re
import  cv2
import  random
import  numpy  as  np

def  image_load(path):
    file_list = os.listdir(path)
    file_name = []
    for  i  in  file_list:
        a = int(  re.sub('[^0-9]', '', i )  ) # 숫자가 아닌것은 '' 로 처리 
        file_name.append(a) 
    file_name.sort()

    file_res = [] 
    for  j  in   file_name:
        file_res.append('%s\\%d.png' %(path,j)  )

    image = []
    for  k  in  file_res:
        img = cv2.imread(k)
        image.append(img)

    return  np.array(image)

def  label_load( path ):
    file = open(path)
    labeldata = csv.reader(file)
    labellist = []
    for  i   in  labeldata:
        labellist.append(i)

    label = np.array(labellist)
    label = label.astype(int)  # 숫자로 변환 
    label = np.eye(2)[label]
    label = label.reshape(-1,2) 
    return  label


def  shuffle_batch( data_list, label ):
    x = np.arange( len( data_list) )
    random.shuffle(x)
    data_list2 = data_list[x]
    label2 = label[x]
    return   data_list2, label2 


def  next_batch( data1, data2, init,  fina ):
    return  data1[ init : fina ],  data2[init : fina] 

#### ● Check train_data and test_data shape

In [13]:
import loader4

train_image='E:\\data11\\train_resize\\'
train_label= 'E:\\data11\\train_label.csv'
test_image='E:\\data11\\test_resize\\'
test_label='E:\\data11\\test_label.csv'

print(loader4.image_load(train_image).shape)
print(loader4.label_load(train_label).shape)
print(loader4.image_load(test_image).shape)
print(loader4.label_load(test_label).shape)

(7000, 32, 32, 3)
(7000, 2)
(470, 32, 32, 3)
(470, 2)


### ■ Train start !

In [2]:
from keras.models import Sequential, save_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
import loader4

batch_size = 28
num_classes = 2
epochs = 100
 
train_image='E:\\data11\\train_resize\\'
train_label= 'E:\\data11\\train_label.csv'
test_image='E:\\data11\\test_resize\\'
test_label='E:\\data11\\test_label.csv'


x_train = loader4.image_load(train_image)
y_train = loader4.label_load(train_label)
x_test = loader4.image_load(test_image)
y_test = loader4.label_load(test_label)      


print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_train.shape[1:])

#(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# One hot Encoding



model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
 
hist = model.fit(x_train, y_train, validation_data=(x_test, y_test), nb_epoch=epochs, batch_size=batch_size, verbose=1)
 
scores = model.evaluate(x_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
 
save_model(model, "E:\data11\\models\\result2.h5")

(7000, 32, 32, 3)
(7000, 2)
(470, 32, 32, 3)
(470, 2)
(32, 32, 3)


C:\anaconda3\envs\tf2.0-gpu\lib\site-packages\ipykernel_launcher.py:74: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 7000 samples, validate on 470 samples
Epoch 1/100
7000/7000 [==============================] - 4s 559us/step - loss: 0.8226 - accuracy: 0.5937 - val_loss: 0.6686 - val_accuracy: 0.6149
Epoch 2/100
7000/7000 [==============================] - 3s 379us/step - loss: 0.6839 - accuracy: 0.6143 - val_loss: 0.6637 - val_accuracy: 0.6085
Epoch 3/100
7000/7000 [==============================] - 3s 385us/step - loss: 0.6481 - accuracy: 0.6410 - val_loss: 0.6345 - val_accuracy: 0.6277
Epoch 4/100
7000/7000 [==============================] - 3s 389us/step - loss: 0.6367 - accuracy: 0.6406 - val_loss: 0.6656 - val_accuracy: 0.6340
Epoch 5/100
7000/7000 [==============================] - 3s 387us/step - loss: 0.6219 - accuracy: 0.6520 - val_loss: 0.6433 - val_accuracy: 0.6170
Epoch 6/100
7000/7000 [==============================] - 3s 434us/step - loss: 0.6184 - accuracy: 0.6591 - val_loss: 0.6309 - val_accuracy: 0.6404
Epoch 7/100
7000/7000 [==============================] - 3s 415us/step 

## ■ Vgg16.ver

In [1]:
import os, cv2, random

# 필요한 라이브러리 import 하는 코드 
import numpy as np

from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation, MaxPooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K   # 백엔드가  텐서 플로우로 되어있어서 
                                     # 텐서 플로우 명령어 필요할 때 tf 대신에 
                                     # k 를 쓰겠다라는 의미 

import loader4

NUM_CLASSES = 2

train_image='E:\\data11\\train_resize\\'
train_label= 'E:\\data11\\train_label.csv'
test_image='E:\\data11\\test_resize\\'
test_label='E:\\data11\\test_label.csv'

print(loader4.image_load(train_image).shape)
print(loader4.label_load(train_label).shape)
print(loader4.image_load(test_image).shape)
print(loader4.label_load(test_label).shape)

x_train = loader4.image_load(train_image)
y_train = loader4.label_load(train_label)
x_test = loader4.image_load(test_image)
y_test = loader4.label_load(test_label)

# 모델 만들기
input_layer = Input((32,32,3))

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size = 2, strides = 2)(x)

x = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size = 2, strides = 2)(x)

x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size = 2, strides = 2)(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size = 2, strides = 2)(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size = 2, strides = 2)(x)

# 완전 연결 계층
x = Flatten()(x)

x = Dense(4096)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Dense(4096)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Dense(NUM_CLASSES)(x)
output_layer = Activation('softmax')(x)

model = Model(input_layer, output_layer)

# convolution 층 5개, 완전 연결계층 3개인 8층 신경망
model.summary()

# 모델 훈련

model_exists = False


if model_exists:
    
    
    model.load_weights('E:\data11\\models\\resultvgg16.h5')
    
    CLASSES = np.array(['정상', '비정상' ]) 
    preds = model.predict( x_test[1:2] )   
    preds_value = CLASSES[ np.argmax( preds, axis= -1 ) ] 
    actual_value = CLASSES[ np.argmax( y_test [1:2], axis = -1) ] 
    print ( preds_value)
    print ( actual_value)
    
else:
        
    opt = Adam(lr=0.0005)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    model.fit(x_train
              , y_train
              , batch_size=28
              , epochs=50
              , shuffle=True
               , validation_data=(x_test,y_test) )
    
    
    model.save_weights('E:\data11\\models\\resultvgg16.h5')
    
    model.evaluate(x_test, y_test, batch_size=28)
    
    CLASSES = np.array(['정상', '비정상' ]) 
    preds = model.predict( x_test[1:2] )   
    preds_value = CLASSES[ np.argmax( preds, axis= -1 ) ] 
    actual_value = CLASSES[ np.argmax( y_test[1:2], axis = -1) ] 
    print ( preds_value)
    print ( actual_value)

Using TensorFlow backend.


(7000, 32, 32, 3)
(7000, 2)
(470, 32, 32, 3)
(470, 2)
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
leaky

7000/7000 [==============================] - 12s 2ms/step - loss: 0.8346 - accuracy: 0.5870 - val_loss: 1.4160 - val_accuracy: 0.6043
Epoch 4/50
7000/7000 [==============================] - 12s 2ms/step - loss: 0.7780 - accuracy: 0.6093 - val_loss: 1.1063 - val_accuracy: 0.6021
Epoch 5/50
7000/7000 [==============================] - 12s 2ms/step - loss: 0.7849 - accuracy: 0.6119 - val_loss: 0.7338 - val_accuracy: 0.6043
Epoch 6/50
7000/7000 [==============================] - 12s 2ms/step - loss: 0.7709 - accuracy: 0.6154 - val_loss: 0.8298 - val_accuracy: 0.6043
Epoch 7/50
7000/7000 [==============================] - 12s 2ms/step - loss: 0.7983 - accuracy: 0.6030 - val_loss: 4.7186 - val_accuracy: 0.4660
Epoch 8/50
7000/7000 [==============================] - 12s 2ms/step - loss: 0.7947 - accuracy: 0.6077 - val_loss: 3.0291 - val_accuracy: 0.6043
Epoch 9/50
7000/7000 [==============================] - 12s 2ms/step - loss: 0.7692 - accuracy: 0.6067 - val_loss: 0.7080 - val_accuracy: 0.5